<a href="https://colab.research.google.com/github/burinexovo/financial-data-web-crawler/blob/main/financial_data_web_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 股市資料蒐集、爬蟲



## 資料爬蟲

## 取得證交所資料
1. 進入證交所網址：https://www.twse.com.tw/zh/index.html
2. 使用開發者模式取得請求資料網址

### 1️⃣ 匯入套件

In [ ]:
# 發送 HTTP 請求以獲取網路上的資料
import requests

# 用來處理和分析結構化數據
import pandas as pd

# 用來處理時間
import datetime as dt

### 2️⃣ 取得證交所上市公司個股日成交資訊

In [ ]:
# "聯華電子"股票代號
stock_id = '2303'

# 獲取今天的日期（型別：str）
date = dt.date.today().strftime("%Y%m%d")
# date = 20241101

print('今日日期：', date)

# 取得證交所網站資料，帶入網址參數 date：時間，stock_id：股票代號
#https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20240918&stockNo=2303
stock_data = requests.get(f'https://www.twse.com.tw/rwd/zh/ \
            afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}')

# 將結果轉換 JSON 格式
json_data = stock_data.json()

# 將 JSON 格式轉換爲 Dataframe 格式，
# 方便在 Python 中處理結構化資料
df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])

df

今日日期： 20240919


,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/09/02,"35,096,170","1,936,319,833",55.60,55.60,54.80,55.40,-0.10,"10,816"
1,113/09/03,"27,896,571","1,542,192,127",55.20,55.60,55.00,55.10,-0.30,"7,645"
2,113/09/04,"85,748,502","4,574,666,003",53.00,54.10,52.30,53.30,-1.80,"31,144"
3,113/09/05,"37,783,857","2,041,606,336",53.90,54.60,53.60,53.90,+0.60,"10,951"
4,113/09/06,"24,617,263","1,337,850,400",54.30,54.50,53.60,54.50,+0.60,"8,368"
5,113/09/09,"46,437,090","2,480,087,923",52.90,53.80,52.60,53.80,-0.70,"12,827"
6,113/09/10,"34,262,285","1,824,628,704",53.80,53.80,52.90,53.30,-0.50,"13,169"
7,113/09/11,"24,122,229","1,298,680,795",53.30,54.00,53.20,54.00,+0.70,"7,614"
8,113/09/12,"35,327,391","1,946,021,545",55.10,55.40,54.90,55.20,+1.20,"16,058"
9,113/09/13,"30,635,902","1,672,171,622",55.20,55.30,54.30,54.60,-0.60,"13,103"


### week2上課後補充：抓取特定某段時間區間 9/10~9/15
為了比較時間區間，必須要把「日期」欄位轉圜為 Pandas 中的 datetime 格式

In [ ]:
def convert_minguo_to_ad(date_str):
    # 113/09/10 透過「/」分割成清單 [113, 09, 10]
    # 分配給變數 year=113, month=09, day=10
    year, month, day = date_str.split('/')
    ad_year = int(year) + 1911  # 將民國年轉換為西元年
    return f"{ad_year}/{month}/{day}"

# 所有民國日期都轉西元日期 e.g.113/09/10 -> 2024/09/10
df['日期'] = df['日期'].apply(convert_minguo_to_ad)

In [ ]:
# 轉換為 pandas 中的 datetime 格式
df['日期'] = pd.to_datetime(df['日期'], format='%Y/%m/%d')

start_date = '2024/09/10'
end_date = '2024/09/15'

# 篩選日期
filtered_df = df[(df['日期'] >= start_date) & (df['日期'] <= end_date)]

print(filtered_df)

          日期        成交股數           成交金額    開盤價    最高價    最低價    收盤價   漲跌價差  \
6 2024-09-10  34,262,285  1,824,628,704  53.80  53.80  52.90  53.30  -0.50   
7 2024-09-11  24,122,229  1,298,680,795  53.30  54.00  53.20  54.00  +0.70   
8 2024-09-12  35,327,391  1,946,021,545  55.10  55.40  54.90  55.20  +1.20   
9 2024-09-13  30,635,902  1,672,171,622  55.20  55.30  54.30  54.60  -0.60   

     成交筆數  
6  13,169  
7   7,614  
8  16,058  
9  13,103  


### 3️⃣ 取得連續月份資料
以個股本益比為例

In [ ]:
# 設定查找時間
date_list = ['20240701', '20240801', '20240901']

all_df = pd.DataFrame()

# 使用迴圈抓取連續月份資料
for date in date_list:
    # 取得證交所網站資料，帶入網址參數 date：時間，stock_id：股票代號
    # https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20240918&stockNo=2303
    # https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU?date=20240303&stockNo=2303
    url = f'https://www.twse.com.tw/rwd/zh/afterTrading/\
      BWIBBU?date={date}&stockNo={stock_id}'

    try:

        # 取得證交所網站資
        stock_data = requests.get(url)

        # 將結果轉換 JSON 格式
        json_data = stock_data.json()

        # 將 JSON 格式轉換爲 Dataframe 格式，
        # 方便在 Python 中處理結構化資料
        df = pd.DataFrame(data=json_data['data'],
                          columns=json_data['fields'])

        # 添加新數據
        all_df = pd.concat([all_df, df], ignore_index=True)

    except Exception as e:

        # 發生錯誤時，列印錯誤訊息
        print(f'無法取得{date}的資料, 可能資料量不足.')


# 輸出前 5 筆資料
print(all_df)

            日期 殖利率(%)  股利年度    本益比 股價淨值比  財報年/季
0   113年07月01日   5.27   112  12.90  1.89  113/1
1   113年07月02日   5.69   112  11.95  1.75  113/1
2   113年07月03日   5.69   112  11.95  1.75  113/1
3   113年07月04日   5.56   112  12.24  1.79  113/1
4   113年07月05日   5.61   112  12.13  1.77  113/1
5   113年07月08日   5.42   112  12.54  1.83  113/1
6   113年07月09日   5.42   112  12.54  1.83  113/1
7   113年07月10日   5.48   112  12.40  1.81  113/1
8   113年07月11日   5.48   112  12.40  1.81  113/1
9   113年07月12日   5.66   112  12.02  1.76  113/1
10  113年07月15日   5.62   112  12.11  1.77  113/1
11  113年07月16日   5.66   112  12.02  1.76  113/1
12  113年07月17日   5.66   112  12.02  1.76  113/1
13  113年07月18日   5.69   112  11.95  1.75  113/1
14  113年07月19日   5.78   112  11.77  1.72  113/1
15  113年07月22日   5.98   112  11.38  1.66  113/1
16  113年07月23日   5.85   112  11.63  1.70  113/1
17  113年07月26日   5.92   112  11.50  1.68  113/1
18  113年07月29日   5.98   112  11.38  1.66  113/1
19  113年07月30日   6.00   112  11.34  1.66

### ⚠️ 如果過度頻繁爬取證交所的資料(約連續50次)會被偵測並封鎖IP，可以搭配time、sleep延遲回應避免被認為是機器人程式。

## 用 BeautifulSoup4 取得 Yahoo 股市資料

###4️⃣ 匯入套件

In [ ]:
# 用來處理時間
from datetime import datetime
# 解析網頁內容
from bs4 import BeautifulSoup

###  5️⃣ 取得當日股價

In [ ]:
def yahoo_stock(stock_id):
    # 取得證交所網站資，帶入網址參數 date：時間，stock_id：股票代號
    url = f'https://tw.stock.yahoo.com/quote/{stock_id}.TW'

    # 使用 requests 取得網頁內容
    response = requests.get(url)

    # 取得 HTML 文本
    html = response.content

    # 使用 Beautiful Soup 解析 HTML 內容
    soup = BeautifulSoup(html, 'html.parser')

    # 使用 find 與 find_all 定位 HTML 的標籤
    # HTML 標籤中的 section id=qsp-overview-realtime-info 下的 time
    time_element = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('time')
    # HTML 標籤中的 section id=qsp-overview-realtime-info 下的 ul li
    table_soups = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('ul')\
                                   .find_all('li')

    # 欄位資料
    fields = []
    # 放資料
    datas = []

    # 使用 for 迴圈取得表格中細部內容
    for table_soup in table_soups:

        # 表格中的 span 標籤
        table_datas = table_soup.find_all('span')

        for num, table_data in enumerate(table_datas):

            if table_data.text == '': # 表格內容空就跳過
                continue

            # 新增資料
            if num == 0:
                fields.append(table_data.text)
            else:
                datas.append(table_data.text)

    # 建立 DataFrame
    df = pd.DataFrame([datas], columns=fields)

    # 增加日期和股號欄位
    df.insert(0,'日期',time_element['datatime'])
    df.insert(1,'股號',stock_id)

    # 回傳 DataFrame
    return df


'''main'''

# "聯華電子"股票代號
stock_id = '2330'

# 呼叫定義函數
yahoo_stock(stock_id)

,日期,股號,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,2024/09/18 14:30,2330,941,945,948,933,940,267.05,947,0.63%,6.00,"28,394","13,595",1.58%


### 6️⃣ 取得季報表資訊


In [ ]:
url = f'https://tw.stock.yahoo.com/quote/{stock_id}/income-statement'
words = url.split('/')
print(words)

k = words[-1]
print(k)

['https:', '', 'tw.stock.yahoo.com', 'quote', '2303', 'income-statement']
income-statement


In [ ]:
# 函數可用於奇摩財報
def url_find(url):

    # 取得 url 的所有 route
    words = url.split('/')

    # 取得最後一個 route
    # 為了知道是損益表、資產負債表、現金流量表
    laset_word = words[-1]

    # 使用requests取得網頁內容
    response = requests.get(url)

    # 取得 HTML 文本
    html = response.content

    # 使用Beautiful Soup解析HTML內容
    soup = BeautifulSoup(html, 'html.parser')

    # 找到表格的表頭 qsp-incom-statment-table
    table_soup = soup.find('section', {'id': 'qsp-{}-table'.format(laset_word)})

    # 找到對應類別標籤
    table_fields=table_soup.find('div', class_='table-header')

    # 解析表頭內容
    table_fields_lines = list(table_fields.stripped_strings)


    # 找到對應的資料
    data_rows = table_soup.find_all('li' ,class_='List(n)')

    # 解析資料行內容
    data = []
    for row in data_rows:
        row_data = list(row.stripped_strings)
        data.append(row_data)

    # 建立 DataFrame
    df = pd.DataFrame(data, columns=table_fields_lines)
    return df

'''main'''

# 抓損益表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/income-statement'
# 抓資產負債表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/balance-sheet'
# 抓現金流量表
url = f'https://tw.stock.yahoo.com/quote/{stock_id}/cash-flow-statement'

# 抓取季報表資料
df = url_find(url).transpose()

# 資料處理
df.columns = df.iloc[0]
df = df[1:]
df.insert(0, '年度/季別', df.index)
df.columns.name = None
df.reset_index(drop=True, inplace=True)

# 輸出資料後5筆
print(df)

      年度/季別        營業現金流         投資現金流         融資現金流        自由現金流  \
0   2024 Q2  377,668,210  -197,607,330   -90,244,583  180,060,880   
1   2024 Q1  436,311,108  -159,806,991   -71,685,617  276,504,117   
2   2023 Q4  394,829,347  -132,319,502   -75,367,133  262,509,845   
3   2023 Q3  294,645,276  -242,243,223   -38,451,204   52,402,053   
4   2023 Q2  167,247,979  -259,326,076   -26,588,885  -92,078,097   
5   2023 Q1  385,244,745  -272,231,795   -64,487,030  113,012,950   
6   2022 Q4  486,881,904  -342,532,013   -69,831,545  144,349,891   
7   2022 Q3  412,698,167  -284,390,325  -130,406,753  128,307,842   
8   2022 Q2  338,849,429  -275,932,106    19,080,454   62,917,323   
9   2022 Q1  372,169,688  -288,073,791   -19,086,188   84,095,897   
10  2021 Q4  378,199,317  -245,343,292    82,284,018  132,856,025   
11  2021 Q3  318,706,207  -177,020,159   -35,324,719  141,686,048   
12  2021 Q2  187,439,492  -169,750,998    75,030,682   17,688,494   
13  2021 Q1  227,815,706  -244,251